# Probelm Set 1: Data section

This dataset is obtained from the website of Chicago data portal. This information is derived from inspections of restaurants and other food establishments in Chicago from January 1, 2010 to the present (2017/4/15).

City of Chicago also used this data to help prioritize the city's food inspection staff to reduce the public's exposure to foodborne illness. It partnered with Allstate’s Quantitative Research & Analytics department to develop a predictive model to help prioritize the city's food inspection staff to reduce the public's exposure to foodborne illness. In this open-source project on github, their model evaluation calculates individualized risk scores for more than ten thousand Chicagoland food establishments.

Data is collected by inspections that are performed by staff from the Chicago Department of Public Health’s Food Protection Program using a standardized procedure. The results of the inspection are inputted into a database, then reviewed and approved by a State of Illinois Licensed Environmental Health Practitioner (LEHP). 

In [168]:
import pandas as pd
import numpy
import matplotlib.pyplot as plt
%matplotlib inline 

In [181]:
data = pd.read_csv("Food_Inspections.csv")
data[:3]

,Inspection ID,DBA Name,AKA Name,License #,Facility Type,Risk,Address,City,State,Zip,Inspection Date,Inspection Type,Results,Violations,Latitude,Longitude,Location
0,2028365,PHLOUR BAKERY AND CAFE,PHLOUR BAKERY AND CAFE,2522179.0,Bakery,Risk 1 (High),1138 W BRYN MAWR AVE,CHICAGO,IL,60660.0,04/14/2017,License Re-Inspection,Pass,"11. ADEQUATE NUMBER, CONVENIENT, ACCESSIBLE, D...",41.983739,-87.659647,"(41.98373915183195, -87.65964672940271)"
1,1959235,WASABI,WASABI RAMEN,2492241.0,Restaurant,Risk 3 (Low),2101-2115 N MILWAUKEE AVE,CHICAGO,IL,60647.0,04/14/2017,License Re-Inspection,Pass,NaN,41.919522,-87.691587,"(41.91952161427481, -87.6915871617949)"
2,1959234,WASABI,WASABI RAMEN,2488598.0,Restaurant,Risk 1 (High),2101-2115 N MILWAUKEE AVE,CHICAGO,IL,60647.0,04/14/2017,License Re-Inspection,Pass,"16. FOOD PROTECTED DURING STORAGE, PREPARATION...",41.919522,-87.691587,"(41.91952161427481, -87.6915871617949)"


In [182]:
data_ = data.dropna()

In [183]:
data_['Risk'] = data_['Risk'].apply(lambda x: int(str(x)[5]))
data_['Zip'] = data_['Zip'].apply(lambda x: str(x)[:-2])

/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


In [184]:
#table for numerical data
data_.iloc[:, [5,14,15]].describe()

,Risk,Latitude,Longitude
count,115239.000000,115239.000000,115239.000000
mean,1.326391,41.878958,-87.676928
std,0.586120,0.082133,0.059652
min,1.000000,41.644670,-87.914428
25%,1.000000,41.820583,-87.708361
50%,1.000000,41.890944,-87.667042
75%,2.000000,41.939486,-87.634649
max,3.000000,42.021064,-87.525094


In [185]:
#table for categorical data
data_.iloc[:, [4,9,11,12,13]].describe()

,Facility Type,Zip,Inspection Type,Results,Violations
count,115239,115239,115239,115239,115239
unique,358,60,58,6,114231
top,Restaurant,60614,Canvass,Pass,18. NO EVIDENCE OF RODENT OR INSECT OUTER OPEN...
freq,78565,4191,59177,76382,31


In [186]:
data_.groupby(['Results']).count().iloc[:,0]

Results
Fail                  25022
No Entry                261
Not Ready                21
Out of Business          32
Pass                  76382
Pass w/ Conditions    13521
Name: Inspection ID, dtype: int64

In [187]:
def score(x):
    if x == 'Fail':
        return -1
    elif x== 'Pass':
        return 0.5
    elif x == 'Pass w/ Conditions':
        return 0.5
    else:
        return 100
data_['Score'] = data_['Results'].apply(lambda x: score(x))
data_ = data_[data_['Score'] != 100]

/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/ipykernel/__main__.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [150]:
#data_[:10]

In [176]:
a = data_.groupby(['Facility Type'])['Risk', 'Score'].mean().reset_index()

In [207]:
a = data_.groupby(['Facility Type'])['Risk', 'Score'].mean().sort_values(['Risk', 'Score'], ascending = False).reset_index()
#Highest score and lowest risk
a.loc[((a['Risk']==3.0) & (a['Score'] == 0.500000))]['Facility Type'].tolist()

['(convenience store)',
 'BEVERAGE/SILVERWARE WAREHOUSE',
 'CAR WASH',
 'CHINESE HERBS',
 'CONVENIENCE/GAS STATION',
 'CONVNIENCE STORE',
 'Convenience Store',
 'DOLLAR STORE SELLING GROCERY',
 'DRUG/GROCERY STORE',
 'Dollar store',
 'FOOD PANTRY',
 'FROZEN DESSERT PUSHCARTS',
 'GAS STATION STORE',
 'GAS STATION/ GROCERY STORE',
 'GAS STATION/GROCERY',
 'GIFT/CARD SHOP WITH CANDY',
 'GROCERY/LIQUOR STORE',
 'GROCERY/SERVICE GAS STATION',
 'GROCERY/TAVERN',
 'Gas station',
 'HERBAL MEDICINE',
 'HERBAL STORE',
 'HOOKA BAR',
 'MOBILE DESSERTS VENDOR',
 'MOBILE FOOD DESSERTS VENDOR',
 'MOBILE FROZEN DESSERT VENDOR',
 'Mobile Frozen Desserts Vendor',
 'Mobile frozen dessert vendor',
 'Nutrition Store',
 'PREPACKAGE MEAL DISTRIBUTOR (1006 Retail)',
 'PUSH CARTS',
 'ROOM SERVICE',
 'SERVICE BAR/THEATRE',
 'SERVICE GAS STATION',
 'SUMMER FEEDING',
 'TAP room/tavern/liquor store',
 'TAVERN/LIQUOR',
 'TAVERN/RESTAURANT',
 'TEA STORE',
 'VFW HALL',
 'night club',
 'theater',
 'video store',
 'wei

In [209]:
#Highest score and lowest risk
#a.loc[((a['Risk']==1.0) & (a['Score'] == 0.500000))]
#a.iloc[:-10,:]
a[-10:]['Facility Type'].tolist()

['Adult Family Care Center',
 'BREWPUB',
 "CHILDERN'S SERVICES  FACILITY",
 'DAYCARE 2 YRS TO 12 YRS',
 'DINING HALL',
 'SOUP KITCHEN',
 'KIDS CAFE',
 'MAIN KITCHEN',
 'NOT FOR PROFIT',
 'UNLICENSED FACILITY']

In [203]:
ZipDF = data_.groupby(['Zip'])['Risk', 'Score'].mean()

In [153]:
ZipDF[:10]

,Risk,Score
Zip,,
60501,2.666667,-0.500000
60601,1.205711,0.296933
60602,1.265476,0.251786
60603,1.196737,0.278311
60604,1.325407,0.306633
60605,1.251748,0.255245
60606,1.220696,0.299166
60607,1.169503,0.174779
60608,1.375998,0.140884


In [204]:
ZipDF.to_csv("Zip_risk.csv")